## Import libraries and read in list of movie IDs

In [1]:
import imdb
import pandas
import time
import numpy as np

In [7]:
movies_and_ids = pandas.read_csv("CS109B_FinalProject-CurrentEDA/EDA_movies.csv", names=["movie", "id"])

In [8]:
movies_and_ids.head()

,movie,id
0,Tempus fugit,tt0390545
1,Sanam,tt0254749
2,Bike Week Exposed: Saints and Sinners,tt0367544
3,Dancer in the Dark,tt0168629
4,My Life Without Me,tt0314412


## Create IMDB object and test out connection

In [9]:
ia = imdb.IMDb()

In [10]:
s_result = ia.search_movie('The Princess Bride')

# Retrieves default information for the first result (a Movie object).
the_unt = s_result[0]
ia.update(the_unt)

# Print some information.
print the_unt['runtime']
print the_unt['rating']
director = the_unt['director'] # get a list of Person objects.


[u'98']
8.1


In [11]:
s2 = ia.get_movie(s_result[0].movieID)

In [12]:
movies_and_ids['id'][10]

'tt0040789'

## Function to extract data from IMDB

In [11]:
def get_data(offset, interval, q):
    all_results = np.array(['id','title', 'canonical title', 'kind', 'director', 'rating', 'year','votes', 
                         'mpaa','runtimes', 'color info', 'genres','Action', 
                         'Adventure', 'Adult', 'Animation', 'Comedy', 'Crime', 
                         'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
                         'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller',
                         'War', 'Western'])
                        
    all_genres = ['Action', 'Adventure', 'Adult', 'Animation',
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Thriller',
    'War', 'Western']

    for num,each in enumerate(movies_and_ids['id'][offset:offset+interval]):
        try:
            result = ia.get_movie(each[3:])
            if (num+1)%5==0: print num
            results = [each]
            for index, each in enumerate(['title', 'canonical title', 'kind', 
                                          'director', 'rating', 'year','votes', 
                                          'mpaa','runtimes', 'color info', 'genres']):
                try:
                    test = result[each]
                    if each in ['genres','director', 'runtimes']: results.append(str(test[0]))
                    else: results.append(str(test))
                except:
                    results.append("null")
            for index, each in enumerate(all_genres):
                try:
                    genre_result = result['genres']
                    if each in genre_result:
                        results.append(1)
                    else:
                        results.append(0)
                except:
                    results.append("null")
            all_results = np.vstack((all_results, results))
        except:
            continue
    features_df = pandas.DataFrame(data=all_results[1:,],  columns=all_results[0,])
    q.put(features_df)

## Use two processes to speed up the extraction

In [12]:
from multiprocessing import Process, Queue

if __name__ == '__main__':
    q = Queue()
    for offset in [0,1000]:
        interval = 1000
        Process(target=get_data, args=(offset,interval,q)).start()


4
4
9
9
14
14
19
19
24
24
29
34
29
39
34
44
39
44
49
49
54
54
59
59
64
64
69
69
74
74
79
79
84
84
89
89
94
94
99
99
104
104
109
109
114
114
119
119
124
124
129
129
134
134
139
139
144
144
149
149
154
154
159
159
164
164
169
169
174
174
179
179
184
184
189
189
194
194
199
199
204
204


2017-04-04 16:29:59,797 WARNING [imdbpy.parser.http.urlopener] /home/cs109b/.local/lib/python2.7/site-packages/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt0321320/combined: Not Found (headers: Date: Tue, 04 Apr 2017 20:49:15 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYq3avOo5MArQbC310u5puZPH0xZIW80AZYEjFL5VQWaKT6sRTR4VlGLqhqhYU7FbX3XMRKTjfiYG4uTnrG5AzPPvAixNQhbv-PqtIga2bb_Nc;path=/;domain=.imdb.com
Set-Cookie: uu=BCYjV-og9cBSrO0oFjx34DRG1X4xZIW80AZYEjFL5VQWaKSQA4jDYDp4we0W1K7jU6pXzH_vvWk6H4RwCwdj6K7aWsr5Zn-TTV3Xi03PWfkCTbJ4YM0q-ay12FK7a59X9zle1J9V9CG3giZeEKnXUYv262vtlkbilE4gP0-Wsov0jD_UeX5u5-mXLWthRsRrMnFlloF5CJDzZy2BKImr8vl-tRFWNsJ70JlLHJOfvC2_C0p8XKVfrqugz3uYYNbkfmCA6W9AfWfdid6SSQCetuPnz2AvEZg0UXe1eTdeF71aeKzvh8Tu5djU-2rki7AYaV1KxGsXRtDjZXxepkwxCCmH4A;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.com
Set-Cookie: cache=BCYutiQTx2EEEcdMq0_JnVHmA

209
209
214
214
219
219
224
224
229
229
234
234
239
239
244
244
249
249
254
254
259
259
264
264
269
269
274
274
279
279
284
289
284
289
294
294


2017-04-04 16:32:54,714 WARNING [imdbpy.parser.http.urlopener] /home/cs109b/.local/lib/python2.7/site-packages/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt0120781/combined: Not Found (headers: Date: Tue, 04 Apr 2017 20:52:10 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYgxUC3JXy4Pf0smictCYLxDFyZG4TEpAw2T6EswYS54hoPW3ZjpiniaKxI1mPUQk1dLTd9g-kyQSbRs__VqO2w6M_0eSd03m8Pwu3N8Hp404E;path=/;domain=.imdb.com
Set-Cookie: uu=BCYjFu1ErfHJHtCJ4HJ-blbAN12ZG4TEpAw2T6EswYS54hoHR4tiYlYZyJfCobs9cg893HS4cPUaa_Wvx2EuNthW6zd6k3K7Ooqc76rhR43a5bu-p2OV41G6AOz2xEOMAyio4jvq1XvAePBsDXgVXj6OrdFRuhB627e4sYLWaME4qpy9B8oMKiDM3QPeA5Aqg-M5KUlaTLbci1wACcDFRHttPsVTtDYnNXe0lFLT7mLkimDCjb9Nj7VZ-ZI-F5l6MQ6d-bTatb3uXdPo5t1AUw0dtQUZV6iKfeRIIx_WGxDrLZDt7Ck-CF8Z2ypYZF6284I5uCONK-P7_QfbUVsQztlrpA;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.com
Set-Cookie: cache=BCYp2YbwOXEBfoYCDLcqOH-17

299
304
299
309
304
309
314
319
314
324
319
329
334
324
339


...

954


2017-04-04 16:51:13,422 WARNING [imdbpy.parser.http.urlopener] /home/cs109b/.local/lib/python2.7/site-packages/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt0157654/plotsummary: Not Found (headers: Date: Tue, 04 Apr 2017 21:10:28 GMT
Server: Server
X-Frame-Options: SAMEORIGIN
Content-Security-Policy: frame-ancestors 'self' imdb.com *.imdb.com *.media-imdb.com withoutabox.com *.withoutabox.com amazon.com *.amazon.com amazon.co.uk *.amazon.co.uk amazon.de *.amazon.de translate.google.com images.google.com www.google.com www.google.co.uk search.aol.com bing.com www.bing.com
Content-Type: text/html;charset=UTF-8
Content-Language: en-US
Vary: Accept-Encoding,User-Agent
P3P: policyref="http://i.imdb.com/images/p3p.xml",CP="CAO DSP LAW CUR ADM IVAo IVDo CONo OTPo OUR DELi PUBi OTRi BUS PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA HEA PRE LOC GOV OTC "
Connection: close
)


959
959
964
964
969
969


2017-04-04 16:51:34,000 WARNING [imdbpy.parser.http.urlopener] /home/cs109b/.local/lib/python2.7/site-packages/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt0236968/combined: Not Found (headers: Date: Tue, 04 Apr 2017 21:10:49 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYsogAD4sXeCdJmgj2wLonL_BVROd_JzlOFsmTys5NmVqNl_ZTkHtnmezrmCJa__TmCGLSSEPbNiy_pHoNnNz4puJgqAXoypZ6Z_GAJxespWmQ;path=/;domain=.imdb.com
Set-Cookie: uu=BCYvCG5_a5c5ql5tu_O9IEY5PRZROd_JzlOFsmTys5NmVqO1Czpvqu94WV65X-cIe4YzJ36-ixw_n_JKuWbu3hi080LoLyjXnc81Paduxu2eG2sWjfx0xQRbEGgCIixd-spBhTDW_ebMZbwPLdrOy4dS6uCq-Cq3XzH4f5Grpe-oVRba7uHq6nV8wzHEj4D8aR5Wlib2wLMsnS6nBtfaxcjLFpk_glwCqWsANjtQOWo2JKMM7TR6tCtTNp7KgfDjk4UuuoNlxqok8edGcJUiM6fYNKw_mwWw2s_wDRpgTNKRXy53J8N3pcCVzwemg9aLMzyRfBB7PH0mfmVIpGkf5L8JRQ;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.com
Set-Cookie: cache=BCYim0RWAYxv16Vxaqh5Nx3Ap

974
974
979
979
984
984
989
989
994
999
994
999


## Collect dataframes from the q and combine

In [13]:
all_dfs = []
for _ in range(2):
    all_dfs.append(q.get())

In [15]:
output_result = all_dfs[0]
for each in all_dfs[1:]:
    output_result = output_result.append(each,ignore_index=True)

## Check data and export to csv

In [16]:
output_result.head()

,id,title,canonical title,kind,director,rating,year,votes,mpaa,runtimes,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
0,tt0179099,null,null,movie,Haroldo Marinho Barbosa,5.5,1986,43,null,110,...,0,0,0,0,0,0,0,0,0,0
1,tt0402362,Margam,Margam,movie,Rajiv Vijayaraghavan,8.4,2003,11,null,India:108,...,null,null,null,null,null,null,null,null,null,null
2,tt0156075,The Shrunken City,"Shrunken City, The",video movie,Ted Nicolaou,4.5,1998,163,Rated PG for fantasy action violence,USA:90,...,0,0,0,0,0,1,0,0,0,0
3,tt4094088,Sweet as You Are,Sweet as You Are,episode,Angela Pope,7.2,1988,53,null,null,...,0,0,0,0,0,0,0,0,0,0
4,tt0494546,The Ranch Girl,"Ranch Girl, The",movie,Allan Dwan,null,1911,null,null,null,...,0,0,0,0,0,0,1,0,0,1


In [22]:
output_result.to_csv("imdb_features_sample_2.csv")

### possible keys we can get from IMDB
title; string; the "usual" title of the movie, like "The Untouchables".
long imdb title; string; "Uncommon Valor (1983/II) (TV)"
canonical title; string; the title in the canonical format,
                         like "Untouchables, The".
long imdb canonical title; string; "Patriot, The (2000)".
year; string; the year of release or '????' if unknown.
kind; string; one in ('movie', 'tv series', 'tv mini series', 'video game',
                      'video movie', 'tv movie', 'episode')
imdbIndex; string; the roman number for movies with the same title/year.
director; Person list; a list of director's name (e.g.: ['Brian De Palma'])
cast; Person list; list of actor/actress, with the currentRole instance
                   variable set to a Character object which describe his
                   role/duty.
cover url; string; the link to the image of the poster.
writer; Person list; list of writers ['Oscar Fraley (novel)']
plot; list; list of plots and authors of the plot.
rating; string; user rating on IMDb from 1 to 10 (e.g. '7.8')
votes; string; number of votes (e.g. '24,101')
runtimes; string list; in minutes ['119'] or something like ['USA:118',
          'UK:116']
number of episodes; int; number or episodes for a series.
color info; string list; ["Color (Technicolor)"]
countries; string list; production's country ['USA', 'Italy']
genres; string list; one or more in (Action, Adventure, Adult, Animation,
		Comedy, Crime, Documentary, Drama, Family, Fantasy, Film-Noir,
		Horror, Musical, Mystery, Romance, Sci-Fi, Short, Thriller,
		War, Western) and other genres defined by IMDb.
akas; string list; list of aka for this movie
languages; string list; list of languages
certificates; string list; ['UK:15', 'USA:R']
mpaa; string; the mpaa rating
episodes (series only); dictionary of dictionary; one key for every season,
                        one key for every episode in the season.
number of episodes (series only); int; total number of episodes.
number of seasons (series only); int; total number of seasons.
series years (series only); string; range of years when the series was produced.
episode of (episode only); Movie object; the parent series for an episode.
season (episode only); int; the season number.
episode (episode only); int; the number of the episode in the season.
long imdb episode title (episode only); string; episode and series title.
series title; string.
canonical series title; string.